In [117]:
import pandas as pd
import json
import numpy as np
import geopy.distance
import geojson
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from sklearn.neighbors import NearestNeighbors
import seaborn as sns

pd.set_option('display.max_columns', 100)

# Data Preparation

## Load data

In [118]:
apartments_train = pd.read_csv('data/apartments_train.csv')
buildings_train = pd.read_csv('data/buildings_train.csv')
apartments_test = pd.read_csv('data/apartments_test.csv')
buildings_test = pd.read_csv('data/buildings_test.csv')

train = pd.merge(
    apartments_train, 
    buildings_train.set_index('id'), 
    how='left', 
    left_on='building_id', 
    right_index=True
)
test = pd.merge(
    apartments_test, 
    buildings_test.set_index('id'), 
    how='left', 
    left_on='building_id', 
    right_index=True
)

## Replace missing coordinates and districts

In [119]:
ind = test[test["longitude"].isna()].index
test.at[ind, "latitude"] = 55.56776345324702
test.at[ind, "longitude"] = 37.48171529826662

In [120]:
ind = train[train.district.isna()].index
train.at[ind, "district"] = 12
ind = test[test.district.isna()].index
test.at[ind, "district"] = 12

## Fixing incorrect coordinates in testset

In [121]:
apartment_coordinates = {
    2511: [55.5438629741104, 37.48233890768276],
    2529: [55.66866253043727, 37.217152651528785],
    4719: [55.63132797843279, 37.426744466015705],
    5090: [55.54390546580924, 37.48229599441532],
    6959: [55.54390546580924, 37.48229599441532],
    8596: [55.54390546580924, 37.48229599441532],
    9547: [55.63399775464791, 37.41982879208156]
}
for k, v in apartment_coordinates.items():
    test.at[k, "latitude"] = v[0]
    test.at[k, "longitude"] = v[1]

# Feature Engineering

## Target features in training set

In [122]:
train["log_price"] = np.log1p(train.price)
train["price_per_m2"] = train.price / train.area_total

## Feature functions

In [123]:
def distance_to_center_feature(data):
    moscow_center = [55.751244, 37.618423]
    coordinates = data[['latitude', 'longitude']].to_numpy()
    dist = [geopy.distance.distance(moscow_center, coordinate).km for coordinate in coordinates]
    data['distance_to_center'] = dist

In [124]:
def distance_to_metro_feature(data, add_metro_lines=True):
    d = data.copy()

    metro_data = pd.read_csv("data/moscow_metro_data.csv", delimiter=";")

    # drop duplicates
    ind = metro_data[metro_data["English transcription"].duplicated()].index
    metro_data = metro_data.drop(ind)
    metro_data = metro_data.reset_index()
    metro_lines = ["metro_line_{}".format(i) for i in range(1, 16)]

    nbrs = NearestNeighbors(n_neighbors=1, algorithm="ball_tree").fit(metro_data[["latitude", "longitude"]])
    distances, indices = nbrs.kneighbors(d[["latitude", "longitude"]])
    d["metro_index"] = indices

    for ind, row in d.iterrows():
        metro_coordinate = metro_data.loc[[row.metro_index]][["latitude", "longitude"]].to_numpy()
        distance = geopy.distance.distance([[row["latitude"], row["longitude"]]], metro_coordinate).km
        data.at[ind, "distance_to_metro"] = distance

        if add_metro_lines:
            number_of_metro_lines = 0
            metro_lines_data = metro_data.loc[[row.metro_index]][metro_lines]
            for metro_line in metro_lines:
                access_to_line = metro_lines_data[metro_line].values[0]
                data.at[ind, metro_line] = access_to_line
                if access_to_line == 1:
                    number_of_metro_lines += 1
            data.at[ind, "number_of_metro_lines"] = number_of_metro_lines

In [125]:
def bearing_feature(data):
    for ind, row in data.iterrows():
        moscow_center = [55.751244, 37.618423]
        c1 = moscow_center
        c2 = [row["latitude"], row["longitude"]]
        y = np.sin(c2[1] - c1[1]) * np.cos(c2[0])
        x = np.cos(c1[0]) * np.sin(c2[0]) - np.sin(c1[0]) * np.cos(c2[0]) * np.cos(c2[1] - c1[1])
        rad = np.arctan2(y, x)
        bearing = ((rad * 180 / np.pi) + 360) % 360
        data.at[ind, "bearing"] = bearing

In [126]:
def distance_to_hospital_feature(data):
    d = data.copy()
    hospital_data = pd.read_csv("data/hospitals.csv")
    hospital_data = hospital_data[["latitude", "longitude"]]
    nbrs = NearestNeighbors(n_neighbors=1, algorithm="ball_tree").fit(hospital_data)
    distances, indices = nbrs.kneighbors(data[["latitude", "longitude"]])
    d["hospital_index"] = indices
    distances = []
    for ind, row in d.iterrows():
        apartment_coordinates = [[row["latitude"], row["longitude"]]]
        hospital_coordinate = hospital_data.loc[[row.hospital_index]].to_numpy()
        distance = geopy.distance.distance(apartment_coordinates, hospital_coordinate).km
        distances.append(distance)
    data["distance_to_hospital"] = distances

In [127]:
def distance_to_universities_feature(data):
    state_uni_coordinates = [55.70444300116007, 37.528611852796914]
    tech_uni_coordinates = [55.76666597872545, 37.68511242319504]
    distances_to_state_uni = []
    distances_to_tech_uni = []
    for ind, row in data.iterrows():
        distances_to_state_uni.append(geopy.distance.distance([row["latitude"], row["longitude"]], state_uni_coordinates).km)
        distances_to_tech_uni.append(geopy.distance.distance([row["latitude"], row["longitude"]], tech_uni_coordinates).km)
    data["distance_to_state_uni"] = distances_to_state_uni
    data["distance_to_tech_uni"] = distances_to_tech_uni

In [128]:
def wealthy_districts_features(data):
    with open("data/geojson/khamovniki_polygon_data.geojson") as f:
        gj_khamovniki = geojson.load(f)
    with open("data/geojson/yakimanka_polygon_data.geojson") as f:
        gj_yakimanka = geojson.load(f)
    with open("data/geojson/arbat_polygon_data.geojson") as f:
        gj_arbat = geojson.load(f)
    with open("data/geojson/presnensky_polygon_data.geojson") as f:
        gj_presnensky = geojson.load(f)
    with open("data/geojson/Tverskoy_polygon_data.geojson") as f:
        gj_tverskoy = geojson.load(f)
    khamovniki_polygon = Polygon(gj_khamovniki["geometries"][0]["coordinates"][0][0])
    yakimanka_polygon = Polygon(gj_yakimanka["geometries"][0]["coordinates"][0][0])
    arbat_polygon = Polygon(gj_arbat["geometries"][0]["coordinates"][0][0])
    presnensky_polygon = Polygon(gj_presnensky["geometries"][0]["coordinates"][0][0])
    tverskoy_polygon = Polygon(gj_tverskoy["geometries"][0]["coordinates"][0][0])
    for ind, row in data.iterrows():
        point = Point([row["longitude"], row["latitude"]])
        data.at[ind, "is_in_khamovniki"] = 1 if khamovniki_polygon.contains(point) else 0
        data.at[ind, "is_in_yakimanka"] = 1 if yakimanka_polygon.contains(point) else 0
        data.at[ind, "is_in_arbat"] = 1 if arbat_polygon.contains(point) else 0
        data.at[ind, "is_in_presnensky"] = 1 if presnensky_polygon.contains(point) else 0
        data.at[ind, "is_in_tverskoy"] = 1 if tverskoy_polygon.contains(point) else 0

In [157]:

def add_box_areas_more_boxes(data):
    """only add values in list to prevent unnecessary slowdown"""
    locs_with_high_corr = [
            'loc_37.575_55.75', 'loc_37.6_55.725', 'loc_37.525_55.75',
           'loc_37.55_55.775', 'loc_37.525_55.725', 'loc_37.575_55.725',
           'loc_37.525_55.7', 'loc_37.5_55.7', 'loc_37.55_55.725',
           'loc_37.475_55.7', 'loc_37.625_55.725', 'loc_37.5_55.75',
           'loc_37.525_55.775', 'loc_37.525_55.675', 'loc_37.55_55.675',
           'loc_37.55_55.75', 'loc_37.4_55.6', 'loc_37.725_55.575',
           'loc_37.925_55.675', 'loc_37.475_55.55', 'loc_37.65_55.575',
           'loc_37.525_55.875', 'loc_37.5_55.55', 'loc_37.475_55.525',
           'loc_37.925_55.7', 'loc_37.6_55.75', 'loc_37.575_55.7', 'loc_37.55_55.7'
    ]
    for lon in np.arange(37.4, 37.95, 0.025):
        for lat in np.arange(55.525, 55.9, 0.025):
            column_name = "loc_" + str(round(lon, 3)) + "_" + str(round(lat, 3))
            if column_name in locs_with_high_corr: #column_name in locs_with_high_corr:
                data[column_name] = 0
                indexes = data[
                    (lon < data['longitude']) & (data['longitude'] < lon + 0.025) & (lat < data['latitude']) & (
                                data['latitude'] < lat + 0.025)
                    ].index

                data.loc[indexes, column_name] = 1

## Feature construction - (NOTE: TAKES SOME TIME)

In [130]:
def construct_features(data):
    distance_to_center_feature(data)
    distance_to_metro_feature(data)
    bearing_feature(data)
    distance_to_hospital_feature(data)
    distance_to_universities_feature(data)
    wealthy_districts_features(data)
    add_box_areas_more_boxes(data)

train_with_feats = train.copy()
test_with_feats = test.copy()
construct_features(train_with_feats)
construct_features(test_with_feats)

# Data Cleaning

## Data imputation

In [131]:
def impute_with_bin_mean(data, train_data, test_data, feature, bin_feature, bins=40, decimals=0, verbose=False):
    tr = train_data.copy()
    te = test_data.copy()
    database = pd.concat([tr, te])
    database["bin"], bins = pd.cut(database[bin_feature], bins=40, retbins=True)
    pb = 0
    for b in bins:
        bin_data = database[(database[bin_feature] >= pb) & (database[bin_feature] <= b)]
        bin_mean = bin_data[feature].mean()
        percentage = (bin_mean / bin_data[bin_feature]).mean()
        d = data[(data[bin_feature] >= pb) & (data[bin_feature] <= b)]
        ind = d[d[feature].isna()].index
        if pd.notna(bin_mean) and len(ind) > 0:
            bin_mean = round(bin_mean, decimals)
            data.at[ind, feature] = bin_mean
            if verbose:
                print(
                    "Set {} for {} rows with {} <= area_living <= {} to {}".format(feature, len(ind), pb, b, bin_mean))
        pb = b


In [133]:
def impute_area_living_bins(train_data, test_data):
    feature = "area_living"
    impute_with_bin_mean(data=train_data, train_data=train_data, test_data=test_data, feature=feature, bin_feature="area_total")
    impute_with_bin_mean(data=test_data, train_data=train_data, test_data=test_data, feature=feature, bin_feature="area_total")
    d = train_data[train_data.area_living.isna()]
    for ind, row in d.iterrows():
        train_data.at[ind, feature] = 0.85 * row.area_total
    d = test_data[test_data.area_living.isna()]
    for ind, row in d.iterrows():
        test_data.at[ind, feature] = 0.85 * row.area_total
    print("Remaining nan values in {} for {}: {}".format("train_data", feature, len(train_data[train_data[feature].isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", feature, len(test_data[test_data[feature].isna()])))
    return train_data, test_data
def impute_area_kitchen_bins(train_data, test_data):
    feature = "area_kitchen"
    impute_with_bin_mean(data=train_data, train_data=train_data, test_data=test_data, feature=feature, bin_feature="area_total")
    impute_with_bin_mean(data=test_data, train_data=train_data, test_data=test_data, feature=feature, bin_feature="area_total")
    d = train_data[train_data.area_kitchen.isna()]
    for ind, row in d.iterrows():
        train_data.at[ind, feature] = 0.05 * row.area_total
    d = test_data[test_data.area_kitchen.isna()]
    for ind, row in d.iterrows():
        test_data.at[ind, feature] = 0.05 * row.area_total
    print("Remaining nan values in {} for {}: {}".format("train_data", feature, len(train_data[train_data[feature].isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", feature, len(test_data[test_data[feature].isna()])))
    return train_data, test_data


def impute_constructed_bins(train_data, test_data):
    feature = "constructed"
    d = train_data[train_data.constructed.isna()]
    ind = d[d.new == 1].index
    train_data.at[ind, feature] = 2021
    d = test_data[test_data.constructed.isna()]
    ind = d[d.new == 1].index
    test_data.at[ind, feature] = 2021
    impute_with_bin_mean(data=train_data, train_data=train_data, test_data=test_data, feature=feature, bins=12, bin_feature="district")
    impute_with_bin_mean(data=test_data, train_data=train_data, test_data=test_data, feature=feature, bins=12, bin_feature="district")
    print("Remaining nan values in {} for {}: {}".format("train_data", feature, len(train_data[train_data[feature].isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", feature, len(test_data[test_data[feature].isna()])))
    return train_data, test_data

def impute_material_bins(train_data, test_data):
    feature = "material"
    impute_with_bin_mean(data=train_data, train_data=train_data, test_data=test_data, feature=feature, bins=20, bin_feature="constructed")
    impute_with_bin_mean(data=test_data, train_data=train_data, test_data=test_data, feature=feature, bins=20, bin_feature="constructed")
    print("Remaining nan values in {} for {}: {}".format("train_data", feature, len(train_data[train_data[feature].isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", feature, len(test_data[test_data[feature].isna()])))
    return train_data, test_data

def impute_heating_category(train_data, test_data):
    feature = "heating"
    ind = train_data[train_data.heating.isna()].index
    train_data.at[ind, feature] = 4
    ind = test_data[test_data.heating.isna()].index
    test_data.at[ind, feature] = 4
    print("Remaining nan values in {} for {}: {}".format("train_data", feature, len(train_data[train_data[feature].isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", feature, len(test_data[test_data[feature].isna()])))
    return train_data, test_data



def impute_parking_category(train_data, test_data):
    feature = "parking"
    ind = train_data[train_data.parking.isna()].index
    train_data.at[ind, feature] = 3
    ind = test_data[test_data.parking.isna()].index
    test_data.at[ind, feature] = 3
    print("Remaining nan values in {} for {}: {}".format("train_data", feature, len(train_data[train_data[feature].isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", feature, len(test_data[test_data[feature].isna()])))
    return train_data, test_data


def impute_remaining_nans(train_data, test_data):
    train_data.loc[:, "seller"] = train_data.loc[:, "seller"].fillna(4)
    test_data.loc[:, "seller"] = test_data.loc[:, "seller"].fillna(4)
    train_data.loc[:, "layout"] = train_data.loc[:, "layout"].fillna(3)
    test_data.loc[:, "layout"] = test_data.loc[:, "layout"].fillna(3)
    train_data.loc[:, "condition"] = train_data.loc[:, "condition"].fillna(4)
    test_data.loc[:, "condition"] = test_data.loc[:, "condition"].fillna(4)
    print("Remaining nan values in {} for {}: {}".format("train_data", "seller", len(train_data[train_data.seller.isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", "seller", len(test_data[test_data.seller.isna()])))
    print("Remaining nan values in {} for {}: {}".format("train_data", "layout", len(train_data[train_data.layout.isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", "layout", len(test_data[test_data.layout.isna()])))
    print("Remaining nan values in {} for {}: {}".format("train_data", "condition", len(train_data[train_data.condition.isna()])))
    print("Remaining nan values in {} for {}: {}".format("test_data", "condition", len(test_data[test_data.condition.isna()])))
    feats = ["ceiling", "bathrooms_shared", "bathrooms_private", "windows_court", "windows_street", "garbage_chute", "balconies", "loggias", "phones", "elevator_passenger", "elevator_service"]
    for feat in feats:
        impute_with_bin_mean(data=train_data, train_data=train_data, test_data=test_data, feature=feat, bin_feature="area_total")
        impute_with_bin_mean(data=test_data, train_data=train_data, test_data=test_data, feature=feat, bin_feature="area_total")
        fill_value = train_data[train_data[feat].notna()][feat].mean(axis=0).round()
        train_data.loc[:, feat] = train_data.loc[:, feat].fillna(fill_value)
        fill_value = test_data[test_data[feat].notna()][feat].mean(axis=0).round()
        test_data.loc[:, feat] = test_data.loc[:, feat].fillna(fill_value)
        print("Remaining nan values in {} for {}: {}".format("train_data", feat, len(train_data[train_data[feat].isna()])))
        print("Remaining nan values in {} for {}: {}".format("test_data", feat, len(test_data[test_data[feat].isna()])))
        
    
    train_data.loc[:, "elevator_without"] = train_data.loc[:, "elevator_without"].fillna(0)
    test_data.loc[:, "elevator_without"] = test_data.loc[:, "elevator_without"].fillna(0)
    train_data.loc[:, "new"] = train_data.loc[:, "new"].fillna(0)
    test_data.loc[:, "new"] = test_data.loc[:, "new"].fillna(0)
        
    return train_data, test_data

In [134]:
def prepare_data(train_data, test_data):
    
    ###### data imputation ######
    print("Imputing missing data...")
    train_data, test_data = impute_area_living_bins(train_data, test_data)
    train_data, test_data = impute_area_kitchen_bins(train_data, test_data)
    train_data, test_data = impute_constructed_bins(train_data, test_data)
    train_data, test_data = impute_material_bins(train_data, test_data)
    train_data, test_data = impute_heating_category(train_data, test_data)
    train_data, test_data = impute_parking_category(train_data, test_data)
    train_data, test_data = impute_remaining_nans(train_data, test_data)

    
    return train_data, test_data
    

train_data = train_with_feats.copy()
test_data = test_with_feats.copy()
train_data, test_data = prepare_data(train_data, test_data)

## One-hot encoding categorical features

In [136]:
district_categories = {
    3: "East",
    6: "South-West",
    5: "South",
    4: "South-East",
    0: "Central",
    2: "North-East",
    1: "North",
    8: "North-West",
    7: "West",
    11: "Novomoskovsk",
    10: "Troitsk",
    9: "Zelenograd",
    12: "Other_district"
}
material_categories = {
    0: "Bricks",
    1: "Wood",
    2: "Monolith",
    3: "Panel",
    4: "Block",
    5: "Monolithic_brick",
    6: "Stalin_project",
    7: "Other_material"
}
heating_categories = {
    0: "Central",
    1: "Individual",
    2: "Boiler",
    3: "Autonomous_boiler",
    4: "Other_heating"
}
parking_categories = {
    0: "Ground",
    1: "Underground",
    2: "Multilevel",
    3: "No_parking"
}
seller_categories = {
    0: "Owner",
    1: "Company",
    2: "Agents",
    3: "Developer",
    4: "Other_seller"
}
layout_categoires = {
    0: "Adjacent",
    1: "Isolated",
    2: "Adjacent_isolated",
    3: "Other_layout"
}
condition_categories = {
    0: "Undecorated",
    1: "Decorated",
    2: "Euro_repair",
    3: "Special_design",
    4: "Other_condition"
}
categorical_features = {
    "parking": parking_categories,
    "heating": heating_categories,
    "material": material_categories,
    "district": district_categories,
    "seller": seller_categories,
    "layout": layout_categoires,
    "condition": condition_categories
}

In [137]:
def one_hot_encode_categorical_features(data, remove_org=True):
    for orig_feat, categories in categorical_features.items():
        for k, category in categories.items():
            data[category] = (data[orig_feat] == k).astype(int)
        if remove_org:
            data.drop(orig_feat, axis=1, inplace=True)
one_hot_encode_categorical_features(train_data)
one_hot_encode_categorical_features(test_data)

## Feature selection

In [102]:
features = ['area_total', 'floor', 'rooms', 'ceiling', 'bathrooms_shared',
       'bathrooms_private', 'windows_court', 'windows_street', 'balconies',
       'loggias', 'phones', 'new', 'latitude', 'longitude', 'constructed',
       'stories', 'elevator_without', 'elevator_passenger', 'elevator_service',
       'garbage_chute', 'distance_to_center', 'distance_to_metro',
       'metro_line_1', 'metro_line_2', 'metro_line_3', 'metro_line_4',
       'metro_line_5', 'metro_line_6', 'metro_line_7', 'metro_line_8',
       'metro_line_9', 'metro_line_10', 'metro_line_11', 'metro_line_14',
       'number_of_metro_lines', 'bearing', 'distance_to_hospital',
       'distance_to_state_uni', 'distance_to_tech_uni', 'Ground',
       'Underground', 'Multilevel', 'No_parking', 'Central',
       'Autonomous_boiler', 'Other_heating', 'Bricks', 'Monolith', 'Panel',
       'Other_material', 'North-West', 'Novomoskovsk', 'Owner', 'Company',
       'Agents', 'Developer', 'Other_seller', 'Isolated', 'Other_layout',
       'Undecorated', 'Decorated', 'Euro_repair', 'Special_design',
       'Other_condition'] # RFECV

In [147]:
features = train_data.columns
features = features.drop(["id", "building_id", "street", "address", "price", "log_price", "price_per_m2"], errors="ignore")
features = features.drop(['area_kitchen', 'area_living'])

## Removing outliers

In [148]:
def remove_min_max_outliers(train_data, test_data, features):
    t = train_data.copy()
    test_feat_min_max = pd.DataFrame({"max": test_data.describe().loc["max"], "min": test_data.describe().loc["min"]}).T
    for feature in features:
        max_val = test_feat_min_max[feature]["max"]
        min_val = test_feat_min_max[feature]["min"]
        max_ind = t[t[feature] > max_val].index
        min_ind = t[t[feature] < min_val].index
        t.drop(max_ind, inplace=True)
        t.drop(min_ind, inplace=True)

    print("Samples before: {} | Samples after: {}".format(len(train_data), len(t)))
    return t

train_data = remove_min_max_outliers(train_data, test_data, features)

Samples before: 23285 | Samples after: 23078


# Training

In [150]:
from sklearn.metrics import mean_squared_log_error

def root_mean_squared_log_error(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [151]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

model1 = RandomForestRegressor(
    n_estimators=200,
    criterion='mse',
    n_jobs=2,
    random_state=42,
    max_features="log2",
    bootstrap=False,
    max_depth=18,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=1.6746306849395426e-07
)

model2 = lgbm.LGBMRegressor(
    random_state=42,
    learning_rate=0.1,
    n_estimators=2000,
    boosting_type='gbdt',
    n_jobs=2,
    num_leaves=20,
    min_data_in_leaf=12,
    max_depth=20,
    max_bin=160,
)

model3 = CatBoostRegressor(
    n_estimators=2000,
    learning_rate=0.2,
    l2_leaf_reg=0.0001351899206097861,
    bagging_temperature=8,
    min_data_in_leaf=14,
    thread_count=2,
    depth=7,
    silent=True,
    random_seed=42,
)

model4 = GradientBoostingRegressor(
    learning_rate=0.2,
    n_estimators=100,
    criterion='friedman_mse',
    subsample=0.9,
    max_depth=10,
)

models = {
    "random_forest": model1,
    "lgbm": model2,
    "cat_boost": model3,
    "gbm": model4,
}

model_targets = {
    "random_forest": "log_price",
    "lgbm": "log_price",
    "cat_boost": "log_price",
    "gbm": "log_price",
}

In [152]:
from sklearn.model_selection import KFold
from tqdm import tqdm

def cross_validation(model, k, data, features, target_feature):
    kf = KFold(
        n_splits=k,
        shuffle=True,
        random_state=42
    )
    train_scores = []
    val_scores = []
    
    for i, (train_index, test_index) in tqdm(enumerate(kf.split(buildings_train)), total=k):
        building_val_ids = buildings_train.id.values[test_index]
        train = data[~data.building_id.isin(building_val_ids)]
        val = data[data.building_id.isin(building_val_ids)]
        X_train = train[features].values
        X_val = val[features].values
        y_train = train[[target_feature]].values.ravel()
        y_val = val[[target_feature]].values.ravel()
        y_train_true = train[["price"]].values.ravel()
        y_val_true = val[["price"]].values.ravel()
        
        model.fit(X_train, y_train)
        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)
        
        if target_feature == "log_price":
            train_pred = (np.e ** train_pred) - 1
            val_pred = (np.e ** val_pred) - 1
        elif target_feature == "price_per_m2":
            train_pred = train_pred * train.area_total
            val_pred = val_pred * val.area_total
        else:
            raise Exception("Unknown target feature")
            
        train_rsmle = root_mean_squared_log_error(y_true=y_train_true, y_pred=train_pred)
        val_rsmle = root_mean_squared_log_error(y_true=y_val_true, y_pred=val_pred)
        train_scores.append(train_rsmle)
        val_scores.append(val_rsmle)
    train_scores = np.array(train_scores)
    val_scores = np.array(val_scores)
    print("Train rsmle: {} | Std: {}".format(train_scores.mean(), train_scores.std()))
    print("Val rsmle: {} | Std: {}".format(val_scores.mean(), val_scores.std()))
    return val_scores.mean()

In [153]:
model1_scores = cross_validation(model1, 5, train_data, features, "price_per_m2")
model2_scores = cross_validation(model2, 5, train_data, features, "log_price")
model3_scores = cross_validation(model3, 5, train_data, features, "log_price")
model4_scores = cross_validation(model4, 5, train_data, features, "log_price")
scores = {
    "rf": model1_scores,
    "lgbm": model2_scores,
    "cb": model3_scores,
    "gbm": model4_scores
}

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.92s/it]


Train rsmle: 0.0752276053003107 | Std: 0.0008927849952340296
Val rsmle: 0.21642288010409816 | Std: 0.022130467751712058


  0%|                                                                                                                                                                                                             | 0/5 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12


 20%|███████████████████████████████████████▍                                                                                                                                                             | 1/5 [00:03<00:15,  3.93s/it]

...

Train rsmle: 0.043628036746660395 | Std: 0.0008055537073811641
Val rsmle: 0.20646921095851742 | Std: 0.024254198430964498


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:06<00:00, 13.36s/it]


Train rsmle: 0.02247490970726406 | Std: 0.00043028513267843094
Val rsmle: 0.2105702452872879 | Std: 0.024930094913785433


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:49<00:00, 21.82s/it]

Train rsmle: 0.027249789529888123 | Std: 0.0008217964809809456
Val rsmle: 0.2281358517268775 | Std: 0.02264727813273152


In [154]:
model_scores = pd.DataFrame(scores, index=[0])
model_scores = model_scores.T
model_scores.columns = ["rsmle"]
model_scores

,rsmle
rf,0.216423
lgbm,0.206469
cb,0.210570
gbm,0.228136


# Prediction using stacking with averaging

In [155]:
f_lgbm = features
model1.fit(train_data[f_lgbm], train_data["price_per_m2"])
model2.fit(train_data[f_lgbm], train_data["log_price"])
model3.fit(train_data[f_lgbm], train_data["log_price"])
model4.fit(train_data[f_lgbm], train_data["log_price"])

model1_pred = model1.predict(test_data[f_lgbm])
model2_pred = model2.predict(test_data[f_lgbm])
model3_pred = model3.predict(test_data[f_lgbm])
model4_pred = model4.predict(test_data[f_lgbm])

model1_pred *= test_data.area_total
model2_pred = (np.e ** model2_pred) - 1
model3_pred = (np.e ** model3_pred) - 1
model4_pred = (np.e ** model4_pred) - 1

predictions = [model1_pred, model2_pred, model3_pred, model4_pred]
pred = np.average(predictions, 
    weights = 1 / model_scores['rsmle'] ** 4,
    axis=0
)
submission = pd.DataFrame()
submission['id'] = test_data.id
submission['price_prediction'] = pred
submission.to_csv('submission.csv', index=False)

[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12


# Prediction using stacking with meta model

In [30]:
##### funksjonene må oppdateres #####

In [31]:
def get_oof(clf, train_data, test_data, features, target_feature, k=5):
    """
    Popular function on Kaggle.
    
    Trains a classifier on 4/5 of the training data and
    predicts the rest (1/5). This procedure is repeated for all 5 folds,
    thus we have predictions for all training set. This prediction is one
    column of meta-data, later on used as a feature column by a meta-algorithm.
    We predict the test part and average predictions across all 5 models.
    
    Keyword arguments:
    clf -- classifier
    x_train -- 4/5 of training data
    y_train -- corresponding labels
    x_test -- all test data
    
    """
    oof_train = np.zeros((train_data.shape[0],))
    oof_test = np.zeros((test_data.shape[0],))
    oof_test_skf = np.empty((k, test_data.shape[0]))
    
    kf = KFold(
        n_splits=k,
        shuffle=True,
        random_state=42
    )
    
    buildings = buildings_train[buildings_train.id.isin(train_data.building_id)]
    
    for i, (train_index, test_index) in enumerate(kf.split(buildings)):
        building_val_ids = buildings.id.values[test_index]
        train = train_data[~train_data.building_id.isin(building_val_ids)]
        val = train_data[train_data.building_id.isin(building_val_ids)]
        X_train = train[features].values
        X_val = val[features].values
        y_train = train[[target_feature]].values.ravel()
        y_val = val[[target_feature]].values.ravel()
        y_train_true = train[["price"]].values.ravel()
        y_val_true = val[["price"]].values.ravel()

        clf.fit(X_train, y_train)
        oof_train[test_index] = clf.predict(X_val)
        oof_test_skf[i, :] = clf.predict(test_data[features].values)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [32]:
def ensemble_predict(models, meta_model, X):
    meta_prediction = np.zeros((X.shape[0], len(models)))
    for i, model in enumerate(models):
        meta_prediction[:, i] = model.predict(X)
    return meta_model.predict(meta_prediction)

In [33]:
def train_val_split(train_data, features, target_features, split_ratio=0.2):
    sample_count = round(split_ratio * buildings_train.shape[0])
    buildings = buildings_train.sample(sample_count)
    train = train_data[~train_data.building_id.isin(buildings.id)]
    val = train_data[train_data.building_id.isin(buildings.id)]
    return train, val

In [34]:
tr, te = train_val_split(train_data, features, ["log_price", "price_per_m2", "price"])

model1_oof_train, model1_oof_val = get_oof(model1, tr, te, features, "log_price")
model2_oof_train, model2_oof_val = get_oof(model2, tr, te, features, "log_price")
model3_oof_train, model3_oof_val = get_oof(model3, tr, te, features, "log_price")
model4_oof_train, model4_oof_val = get_oof(model4, tr, te, features, "log_price")

ValueError: shape mismatch: value array of shape (3586,) could not be broadcast to indexing result of shape (1084,)

In [ ]:
x_train = np.concatenate((
    rf_oof_train,
    lgbm_oof_train,
    cb_oof_train,
    gbm_oof_train
), axis=1)

x_val = np.concatenate((
    rf_oof_val,
    lgbm_oof_val,
    cb_oof_val,
    gbm_oof_val
), axis=1)

In [ ]:
META_MODEL = lgbm.LGBMRegressor(
    random_state=SEED,
    learning_rate=0.05,
    n_estimators=200,
    boosting_type='gbdt',
    n_jobs=2,
    num_leaves=30
)

# fit with estimations of training data and decrease the error in predicting the true data
META_MODEL.fit(x_train, y_train)
# predict the test data based on the output of the other models
pred = META_MODEL.predict(x_val)

rsmle = root_mean_squared_log_error(y_true=(np.e ** y_val) - 1, y_pred=(np.e ** pred) - 1)
print(f'Valid rmsle: {rsmle :.4f}')

In [ ]:
pred = ensemble_predict([model1, model2, model3, model4], META_MODEL, X_test)
pred = (np.e ** pred) - 1
submission = pd.DataFrame()
submission['id'] = test_data.id
submission['price_prediction'] = pred
submission.to_csv('submission_stacking.csv', index=False)

# Feature selection experimental